In [1]:
!pip install datasets
!pip install evaluate
!pip install sacrebleu
!pip install rouge_score

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=56e4ca26c40fbfd435d73c2ebbe506cca0d20cb53953a81a4a0308c44c26f85c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef1

# The CNN/DailyMail Dataset

In [2]:
from datasets import load_dataset

In [ ]:
# Load the CNN/DailyMail dataset, version 3.0.0
dataset = load_dataset("cnn_dailymail", "3.0.0")
print(f"Features: {dataset['train'].column_names}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

In [4]:
print("leght of article: ", len(dataset["train"][0]["article"]))
print("lenght of highlight:", len(dataset["train"][0]["highlights"]))
dataset['train'].to_pandas().head(1)[['article', "highlights"]]

leght of article:  2527
lenght of highlight: 217


,article,highlights
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...


# My findings
1. the article are very large
2. pose a issue for the transformer architecture
3. we generally heve a context lenght of about 1000 or so tokens (about 1 or so paragraph)
4. to solve it we have to truncate it to 1000 token s
5. we may be lossing some information which is at the end of the aricles unfortunaitly

# Text Summarization Pipelines


In [5]:
sample_text = dataset['train'][0]['article'][:2000]
summaries = {}

In [6]:
sample_text

'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details o

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

In [5]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
string = "The U.S. are a country. The U.N. is an orgnaization"
sent_tokenize(string)

['The U.S. are a country.', 'The U.N. is an orgnaization']

## summarization Baseline

- baseline are usefull to evaluate our pretraine model to see how good it is by comaring it to the base line as a simpleist evaluation

In [6]:
# A common baseline for summarizatin is to just take top 3 sentence as a summary
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

summaries["baseline"] = three_sentence_summary(sample_text)

NameError: name 'sample_text' is not defined

In [11]:
print(summaries['baseline'])

LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.
Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties.
"I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month.


## GPT-2

In [ ]:
from transformers import pipeline, set_seed
set_seed(42)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [13]:
pipe = pipeline("text-generation", model="gpt2", device=0)
gpt2_query = sample_text+ "\nTL;DR:"
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries["gpt2"] = "\n".join(sent_tokenize(pipe_out[0]['generated_text'][len(gpt2_query):]))


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [14]:
summaries['gpt2']

' He has no plans to buy his own plane tickets and just wants to be seen doing what his father does best when it comes to his kids.'

## T-5

In [15]:
pipe = pipeline("summarization", model="t5-large", device=0)
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]['summary_text']))

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

## BERT

In [16]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn", device=0)
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]['summary_text']))

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

## PEGASUS

In [17]:
pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail", device=0)
pipe_out = pipe(sample_text)
summaries["pegasus"] = pipe_out[0]['summary_text'].replace(" .<n>", ".\n")

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

# Comparing Different Summaries

In [18]:
print("GROUND TRUTH")
print(dataset["train"][0]["highlights"])
print("")
for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

GROUND TRUTH
Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .
Young actor says he has no plans to fritter his cash away .
Radcliffe's earnings from first five Potter films have been held in trust fund .

BASELINE
LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.
Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties.
"I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month.

GPT2
 He has no plans to buy his own plane tickets and just wants to be seen doing what his father does 

In [ ]:
import evaluate
import pandas as pd
import numpy as np

# Load the sacrebleu metric
blue_metrics = evaluate.load("sacrebleu")

# Predictions and references
predictions = ["the the the the the the"]
references = [["the cat is on the mat"]]

# Compute BLEU score with smoothing
results = blue_metrics.compute(predictions=predictions, references=references, smooth_method="floor", smooth_value=0)

# Round precision values and format results in a DataFrame
results['precisions'] = [np.round(p, 2) for p in results['precisions']]

# Create a DataFrame from the results
df_results = pd.DataFrame.from_dict(results, orient='index', columns=['Value'])

print(df_results)


                             Value
score                          0.0
counts                [2, 0, 0, 0]
totals                [6, 5, 4, 3]
precisions  [33.33, 0.0, 0.0, 0.0]
bp                             1.0
sys_len                          6
ref_len                          6


In [20]:
predictions = ["the cat is on mat"]
blue_metrics.compute(predictions=predictions, references=references)

{'score': 57.89300674674101,
 'counts': [5, 3, 2, 1],
 'totals': [5, 4, 3, 2],
 'precisions': [100.0, 75.0, 66.66666666666667, 50.0],
 'bp': 0.8187307530779819,
 'sys_len': 5,
 'ref_len': 6}

# why rouge is used in summarization task?

- rouge is based on recall (from the actual truth how manny we got right)
- for summarization
  - form reference how many the model(generation) got correct
  - this means we are giving importance to the words that are present in the reference text.
- Another way of saying:
  - how many n-grams that are in generation also present in reference
  - *BlEU is the opposit of it i.e. how many for n-grame that are in reference that are present in generation*

In [21]:
import evaluate
rouge = evaluate.load("rouge")
results = rouge.compute(predictions=predictions, references=references)
# Display the results in a DataFrame for better readability
df_results = pd.DataFrame.from_dict(results, orient='index', columns=['Value'])
print(df_results)

              Value
rouge1     0.909091
rouge2     0.666667
rougeL     0.909091
rougeLsum  0.909091


In [22]:
summaries.keys()

dict_keys(['baseline', 'gpt2', 't5', 'bart', 'pegasus'])

In [23]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = evaluate.load('rouge')
records = []
references = [[dataset["train"][0]['highlights']]]
for model_name in summaries.keys():
    result = rouge_metric.compute(predictions=[summaries[model_name]], references=references)
    rouge_dict = dict((rn, result[rn]) for rn in rouge_names)
    records.append(rouge_dict)


In [24]:
pd.DataFrame(records, index=summaries.keys())

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.335484,0.248366,0.296774,0.335484
gpt2,0.179104,0.123077,0.179104,0.179104
t5,0.575342,0.450704,0.547945,0.575342
bart,0.717391,0.511111,0.652174,0.717391
pegasus,0.800000,0.692308,0.800000,0.800000


# Evaluating PEGASUS on the CNN/DailyMail Datase

In [ ]:
def evaluate_summaries_baseline(dataset, metric, column_text="article", column_summary='highlights'):
    predictions = [three_sentence_summary(text) for text in dataset[column_text]]
    references = [[summary] for summary in dataset[column_summary]]
    result = metric.compute(predictions=predictions, references=references)
    return result

In [15]:
test_sampled = dataset['test'].shuffle(seed=42).select(range(1000))

rouge_metric = evaluate.load('rouge')
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

score = evaluate_summaries_baseline(test_sampled, rouge_metric)
rouge_dict = dict((rn, score[rn])for rn in rouge_names)
pd.DataFrame.from_dict(rouge_dict, orient="index", columns=["baseline"]).T

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.389185,0.171622,0.245191,0.354469


In [26]:
from tqdm import tqdm
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"


def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i+batch_size]

def evaluate_summaries_pegesus(
    dataset,
    metric,
    model,
    tokenizer,
    batch_size=16,
    device=device,
    column_text="article",
    column_summary='highlights'
):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in zip(article_batches, target_batches):
        inputs = tokenizer(article_batch, max_length=1024, truncation=True,padding=True, return_tensors="pt").to(device)
        predictions = model.generate(
            input_ids=inputs["input_ids"].to(device),
            attention_mask=inputs["attention_mask"].to(device),
            num_beams=5,
            length_penalty=0.8,
            max_length=128
        )
        decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True)

        print(metric.compute(predictions=decoded_predictions, references=target_batch))
        break



In [20]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")
model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-cnn_dailymail").to(device)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
evaluate_summaries_pegesus(
    test_sampled,
    rouge_metric,
    model,
    tokenizer)

OutOfMemoryError: CUDA out of memory. Tried to allocate 320.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 169.06 MiB is free. Process 3952 has 14.58 GiB memory in use. Of the allocated memory 14.28 GiB is allocated by PyTorch, and 185.60 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)